# LEARNING WITH ERRORS

### LWE Background

In [1]:
print('Oded Regev')

Oded Regev


### LWE Intro

https://cims.nyu.edu/~regev/papers/lwesurvey.pdf

https://web.archive.org/web/20231008081450/https://cryptopedia.dev/posts/kyber/

https://cims.nyu.edu/~regev/papers/qcrypto.pdf

LWE is computational problem of learning a linear function $f(A)$ which takes values over a ring, given many noisy samples of the function. These samples look like the pair $A, A\cdot S+e$ where $S$ is the secret element which defines the linear function, $e$ is some small error term from a known distribution, and $A$ is also known element of the ring 

Cryptosystems based on LWE have these features: 
1. They use modular arithmetic under two different moduli, the plaintext modulus and the ciphertext modulus 
2. The secret key is an element of a vector space $\bmod N$ 
3. Messages are encrypted by adding an encoded noisy message to a large dot-product

The noisy message is properly encoded sum of the message and a small error or noise term. 

The dot-product is between the secret and a random element of the vector space, where this is random element is provided as part of the ciphertext 

This looks like $(A, A \cdot s +$ encoded $(m,e))$ , where $A$ is an element of the vector space

If the secret key is known then the dot product can be subtracted out, leaving only the encoded message and noise. Thanks to the special way in which the message and noise are encoded, the noise can be remove from the encoding, leaving only the message behind. 

There are two common ways to store message and noise in LWE systems: you can either store the message in the high bits of the LWE sample and the noise in the low bits or vice-versa. 

What algorithm could we use if there was no error term? 

In [1]:
print('Gaussian elimination')

Gaussian elimination


### LWE High Bits Message

Parameters: 
1. vector space dimension $n$ 
2. ciphertext modulus $q$ 
3. plaintext modulus $p$ (can only encrypt messages $<p$)
4. scaling factor $\Delta = round(q/p)$

Key-gen: 
- The secret key $S$ is a random element of the vector space $Z_q^n$ 

Ciphertext format: 
- Ciphertexts consist of pair $A,b$ where $A$ is an element of the vector space $Z_q^n$ and $b$ is an element of $Z_q$ 

Encryption given message $m$: 
1. Sample $A$, a random element of the vector space $Z_q^n$
2. Sample the error term $e$, an integer in the range $[-\frac{\Delta}{2}, \frac{\Delta}{2}]$, can be Gaussian distribution or uniform
3. Compute $b = $A \cdot S + \Delta m + e$ 
4. Return the pair $(A,b)$

Decryption given ciphertext $(A,b)$: 
1. Compute $x = b - A \cdot S \bmod q$ and then interpret x as an integer (not $\bmod q$)
2. Compute $m = round(\frac{x}{\Delta})$ where rounding and division happen over integers 
3. return m 

In this system, decryption works because after the dot product is removed, the remaining equation for the noisy message $\Delta m +e$ holds over integers, so there is no modular wraparound due to $q$. As such, removing the error term can be done by rounding integer-division. This requires the parameters to be chosen so that $\Delta m + e < \frac{q}{2}$ holds. 

In [2]:
print(201)

201


### LWE Low Bits Message

$ q,p$ must be coprime 

Encryption given message $m$: 
1. Sample $A$, a random element of the vector space $Z_q^n$ 
2. Sample the error term $e$, an integer in the range $[-\frac{q}{2p},\frac{q}{2p}]$
3. Compute $b = A \cdot S + pe$
4. Return pair $(A,b)$

Decryption given ciphertext $(A,b)$: 
1. Compute $x = b - A \cdot S$ centered $\bmod q$ and then interpret $x$ as integer (not $\bmod q$). Note this centered modular reduction must produce a result between $(-q/2, q/2]$, as opposed to usual modular reduction producing a result between $[0, q-1]$
2. Compute $m = x \bmod p$ where division and rounding happens over the integers 
3. return $m$ 

Decryption works because after removing the dot product, the remaining equation for the noisy message $ m +pe$ holds over the integers, so there is no modular wraparound due to $q$. As such, removing the error term $e$ can be done by reducing $\bmod p$. This requires parameters to be chosen so that $m + pe < \frac{q}{2}$ holds. 

In [3]:
print(147)

147


### From Private to Public Key LWE

This can be turned into a public key cryptosystem by using the **additively homomorphic" property of LWE. That is, given an encryption $<A,b>$ encrypting the number $m$, it is possible for anyone to turn this into a valid encryption of $m + m_2$ for any number $m_2$. Explicitly this is done by computing $<A, b+m_2>$ for low-bit message and $<A,b+ \delta m_2>$ for high bit messages. 

Similarly, adding two LWE ciphertexts produces a new valid ciphertext which encrypts the sum of corresponding plaintexts. The owner of the private key can use this property to turn LWE into a public key system by releasing many different encryptions of zero as the public key. For Alice to use this information to encrypt, she first chooses a random subset of these encryptions of zero and add them together. By the second additively homomorphic property, this is also a valid encryption of zero. Next, Alice creates a new encoding for her message $m$, and adds this encoded message to this new encryption of zero. By the first additevly homomoprhic property, this is a valid encryption fo $m$. This procedure requires the distribution from which the noise samples are drawn to be carefully chosen so that the final error term is (with high probability) below the threshold needed to decrypt. 

In order for this to be secure, it must be hard for an adversary to determine which public key samples were summed to produce the LWE ciphertext. To ensure this, the number of encryptions of zero in the public key must be significantly larger than the LWE dimesion. As such, the size of public key scales as $O(n^2 log(q))$ bits, making LWE cryptosystems have large public keys. 

What is the size of a Kyber1024 public key in bytes?

In [1]:
print(1568)

1568


### Noise Free

We issue 64 encryptions of 0, that way we get $A_i,b_i$ we stack them up in matrices and solve

$As =b$ to get the secret key $s$ then we use that key do decrypt each byte of encrypted flag

In [32]:
from python import utils
import ast 

n = 64
p = 257
q = 0x10001

F = GF(q) 
V = VectorSpace(F,n)

c = utils.Challenge(int(13411))
c.readline()

A = [] 
b = [] 
for _ in range(n): 
    c.json_send({'option': 'encrypt', 'message': int(0)})
    r = c.json_recv()

    A_i = ast.literal_eval(r['A'])
    A.append(A_i) 

    b_i = int(r['b'])
    b.append(b_i) 

A = matrix(F,A)
b = vector(F,b) 
S = A.solve_right(b) 


flag = list("crypto{????????????????????????}")
for i in range(7,len(flag)-1): 
    c.json_send({'option': 'get_flag','index': int(i)})
    r = c.json_recv()
    A = V(tuple(ast.literal_eval(r['A'])))
    b = F(int(r['b']))
    flag[i]=chr(int(b-A*S))

print(''.join(flag))


crypto{linear_algebra_is_useful}


### Nativity

https://eprint.iacr.org/2020/666.pdf

https://eprint.iacr.org/2013/839.pdf

In [6]:
import numpy as np
from random import SystemRandom

n = 64

m = 512
dtype = np.uint16

random = SystemRandom()
sigma = 2.3
def normal(): return round(random.gauss(0, sigma))
def binary(): return random.randrange(2)

def sample(shape, dist):
    return np.fromfunction(np.vectorize(lambda *_: dist()), shape).astype(dtype)



sample((m,), binary)

array([1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1,